## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-16-03-56-10 +0000,nypost,The Trump economy is far from the Reagan boom—...,https://nypost.com/2025/11/15/business/trumps-...
1,2025-11-16-03-51-27 +0000,nypost,Thousands take to the streets in Mexico City f...,https://nypost.com/2025/11/15/world-news/gen-z...
2,2025-11-16-03-51-02 +0000,missionlocal,Mole to Die For: Mission District chefs cook u...,https://missionlocal.org/2025/11/mole-to-die-f...
3,2025-11-16-03-48-46 +0000,latimes,"Three dead, one wounded in South Gate domestic...",https://www.latimes.com/california/story/2025-...
4,2025-11-16-03-21-29 +0000,nypost,Photographer captures skydiver silhouetted aga...,https://nypost.com/2025/11/15/us-news/astropho...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2332/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,33
259,he,8
33,dead,8
98,california,7
313,epstein,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
65,2025-11-15-19-24-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...,86
104,2025-11-15-15-47-56 +0000,nypost,Trump calls Marjorie Taylor Greene ‘disgrace’ ...,https://nypost.com/2025/11/15/us-news/trump-ca...,71
118,2025-11-15-13-22-06 +0000,cbc,Trump calls it quits with Marjorie Taylor Gree...,https://www.cbc.ca/news/world/u-s-trump-greene...,70
91,2025-11-15-17-07-31 +0000,nyt,Donald Trump Ends Support for Marjorie Taylor ...,https://www.nytimes.com/2025/11/14/us/politics...,63
62,2025-11-15-19-43-03 +0000,bbc,Trump says he will take legal action against B...,https://www.bbc.com/news/articles/c891jp9j79do...,60


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
65,86,2025-11-15-19-24-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...
23,37,2025-11-16-00-29-01 +0000,bbc,My mum was a 17-year-old free spirit - so she ...,https://www.bbc.com/news/articles/cr43vx0rrwvo...
81,30,2025-11-15-17-54-19 +0000,wapo,Former Fed governor violated central bank’s tr...,https://www.washingtonpost.com/business/2025/1...
97,30,2025-11-15-16-45-59 +0000,nypost,Miss Israel getting death threats after being ...,https://nypost.com/2025/11/15/world-news/miss-...
17,27,2025-11-16-01-42-19 +0000,cbc,Anti-government protesters clash with riot pol...,https://www.cbc.ca/news/world/gen-z-protest-me...
11,27,2025-11-16-02-05-00 +0000,latimes,4 dead after panga boat capsizes off Southern ...,https://www.latimes.com/california/story/2025-...
140,27,2025-11-15-10-56-49 +0000,cbc,Vatican returns Indigenous cultural items to C...,https://www.cbc.ca/news/vatican-returns-indige...
98,25,2025-11-15-16-42-57 +0000,nyt,Waymo Was Thriving in San Francisco. Then One ...,https://www.nytimes.com/2025/11/15/us/waymo-sa...
116,24,2025-11-15-13-41-18 +0000,nypost,Let our people go! Zohran Mamdani victory spur...,https://nypost.com/2025/11/15/us-news/staten-i...
41,24,2025-11-15-22-07-12 +0000,nypost,Iran says it seized oil tanker in Strait of Ho...,https://nypost.com/2025/11/15/world-news/iran-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
